In [1]:
import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client['lol']

In [3]:
matchlist_col = db['matchlist']
matchlists = matchlist_col.find(no_cursor_timeout=True)

C:\Users\jungm\AppData\Local\Programs\Python\Python37\lib\site-packages\pymongo\collection.py:1533: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://docs.mongodb.com/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


# Tier

In [4]:
from riotwatcher import LolWatcher, ApiError
from LolCrawler.config import config
import time

watcher = LolWatcher(api_key=config['api_key'])

## get_tier
def get_tier(puuid):
    while True:
        try:
            summoner = watcher.summoner.by_puuid('KR', puuid)
            data = watcher.league.by_summoner('KR', summoner['id'])
            if len(data) > 0:
                data = data[0]
            else:
                return 'U', 0, 0
            break
        except ApiError as e:
            print(e)
            print('Exception has been raised. Sleeping for 5 min')
            time.sleep(300)

    data['tier'][0] # I B S G P D M GM C
    if data['tier'][:2] == 'GO':
        tier = 'G'
    elif data['tier'][:2] == 'GR':
        tier = 'GM'
    else:
        tier = data['tier'][0]
    
    if data['rank'] == 'I':
        rank = '1'
    elif data['rank'] == 'II':
        rank = '2'
    elif data['rank'] == 'III':
        rank = '3'
    elif data['rank'] == 'IV':
        rank = '4'
    
    lp = data['leaguePoints']
    winrate = round(data['wins'] / (data['wins']+data['losses']) * 100, 2)

    if tier in ['C', 'GM', 'M']:
        return tier, lp, winrate
    else:
        return tier+rank, lp, winrate


# Do

In [4]:
puuid = 'QY3_i7hwfkU5n9w0N57pOitmePy1LYp3J8B0nu9Qi-bsHyA0IW6Cq1tfRXW0qBDsfiWh3bkRkgpiJg'

In [5]:
for index, user in enumerate(matchlists):
    if user['_id'] == puuid:
        print(index)
    

20706


In [7]:
i = 11023
while True:
    matchlists = matchlist_col.find(no_cursor_timeout=True)
    try:
        for user in matchlists[i:]:
            puuid = user['_id']
            tier, lp, winrate = get_tier(puuid)
            user['extractions']['tier'] = tier
            user['extractions']['leaguePoints'] = lp
            user['extractions']['winrate'] = winrate
            matchlist_col.replace_one(filter={'_id': puuid}, replacement=user, upsert=False)
    except:
        print(time.strftime('%m-%d %H:%M')+' :: Exception has been raised. Sleeping for 1 min :: puuid =', puuid)
        matchlists = matchlist_col.find(no_cursor_timeout=True)
        for index, user in enumerate(matchlists):
            if user['_id'] == puuid:
                i = index
        time.sleep(60)
        

C:\Users\jungm\AppData\Local\Programs\Python\Python37\lib\site-packages\pymongo\collection.py:1533: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://docs.mongodb.com/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


11-10 22:04 :: Exception has been raised. Sleeping for 1 min :: puuid = QY3_i7hwfkU5n9w0N57pOitmePy1LYp3J8B0nu9Qi-bsHyA0IW6Cq1tfRXW0qBDsfiWh3bkRkgpiJg


In [ ]:
from IPython.display import Audio
Audio('D:\long_success.wav', autoplay=True)

dict